In [1]:
import os
os.chdir("../")

In [2]:
from src.hf_dataset.dataset import get_banking_77

c:\Users\asus\miniconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.getcwd()

'd:\\goldy\\transformers'

In [4]:
ds = get_banking_77()

In [5]:
from src.hf_models.get_model import get_distilbert
model, tokenizer, config = get_distilbert(task="SequenceClassification")

d:\goldy\transformers\models
Model loaded successfully.


In [6]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [7]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
ds = ds.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
ds = ds.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [8]:
from src.config.configuration import ConfigurationManager
config_manager = ConfigurationManager()
params = config_manager.get_distilbert_config()

params

d:\goldy\transformers\models


DistilBert(model_path='d:\\goldy\\transformers\\models\\distilbert\\model', tokenizer_path='d:\\goldy\\transformers\\models\\distilbert\\tokenizer', config_path='d:\\goldy\\transformers\\models\\distilbert\\config', model_name='distilbert')

In [9]:

from adapters import AutoAdapterModel

config.num_labels=77

model = AutoAdapterModel.from_pretrained(
    params.model_path,
    config=config,
)

Some weights of DistilBertAdapterModel were not initialized from the model checkpoint at d:\goldy\transformers\models\distilbert\model and are newly initialized: ['heads.default.0.bias', 'heads.default.0.weight', 'heads.default.2.bias', 'heads.default.2.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.add_adapter("banking77", config="seq_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "banking77",
    num_labels=77)

# Activate the adapter
model.train_adapter("banking77")

In [13]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    compute_metrics=compute_accuracy,
)

In [14]:
trainer.train()

Step,Training Loss
200,2.106700
400,1.359400
600,1.038400
800,0.898600


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=939, training_loss=1.276521237894369, metrics={'train_runtime': 118.5621, 'train_samples_per_second': 253.108, 'train_steps_per_second': 7.92, 'total_flos': 637369366263840.0, 'train_loss': 1.276521237894369, 'epoch': 3.0})

In [16]:
trainer.evaluate(ds["test"])

{'eval_loss': 0.8099204897880554,
 'eval_acc': 0.8116883116883117,
 'eval_runtime': 7.0459,
 'eval_samples_per_second': 437.132,
 'eval_steps_per_second': 13.767,
 'epoch': 3.0}

In [17]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

Device set to use cuda:0
The model 'DistilBertAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gem

[{'label': 'LABEL_59', 'score': 0.12981785833835602}]